# Automated .csv file import into a PostgreSQL database

## Introduction :

### In PostgreSQL, it is necessary to specify the data types of columns before importing data. This can be more cumbersome, especially when dealing with tables with a large number of columns. The requirement to define the data types in advance can make the import process more time-consuming and error-prone, as it involves accurately specifying the data types for each column.
### However, it's important to note that PostgreSQL's approach of explicitly defining data types offers the advantage of data integrity enforcement and stricter control over the data being imported. It ensures that the imported data aligns with the defined table schema, reducing the risk of data inconsistencies or integrity violations.

In [1]:
import os
import numpy as np
import pandas as pd
import psycopg2

h = os.getcwd()

In [18]:
os.listdir(h)

['.condarc',
 '.ipynb_checkpoints',
 '.ipython',
 '.jupyter',
 '.ms-ad',
 '.vscode',
 '1110',
 '3049109',
 '3D Objects',
 'AppData',
 'Application Data',
 'Contacts',
 'Cookies',
 'Customer Contracts.csv',
 'Customer Demo.csv',
 'Customer Engagements.csv',
 'datasets',
 'Desktop',
 'Documents',
 'Downloads',
 'edb_pgagent_pg15.exe',
 'Favorites',
 'IntelGraphicsProfiles',
 'Links',
 'Local Settings',
 'Microsoft',
 'MicrosoftEdgeBackups',
 'Music',
 'My Documents',
 'NetHood',
 'NTUSER.DAT',
 'ntuser.dat.LOG1',
 'ntuser.dat.LOG2',
 'NTUSER.DAT{6633646e-9574-11ec-a3af-e513d6db25ea}.TM.blf',
 'NTUSER.DAT{6633646e-9574-11ec-a3af-e513d6db25ea}.TMContainer00000000000000000001.regtrans-ms',
 'NTUSER.DAT{6633646e-9574-11ec-a3af-e513d6db25ea}.TMContainer00000000000000000002.regtrans-ms',
 'ntuser.ini',
 'OneDrive',
 'Pictures',
 'PostgreSQL.ipynb',
 'PrintHood',
 'Recent',
 'Saved Games',
 'Searches',
 'SendTo',
 'Start Menu',
 'Templates',
 'Untitled.ipynb',
 'Videos']

In [8]:
#find CSV files in my current working directory #isolate only the CSV files

csv_files = []
for file in os.listdir(os.getcwd()):
    if file.endswith('.csv'):
        csv_files.append(file)

In [19]:
#make a new directory
dataset_dir = 'datasets'

#create the bash command to make a new directory
# mkdir dataset_dir
try:
    mkdir = 'mkdir {0}'.format(dataset_dir)
    os.system(mkdir)
except:
    pass

In [24]:
#move the CSV files in the new directory
import shutil
#mv filename directory
for csv in csv_files:
    src_file = os.path.join(h, csv)  # Full path of the source file
    dst_file = os.path.join(dataset_dir, csv)  # Full path of the destination file
    shutil.move(src_file, dst_file)
    print(f"Moved {csv} to {dst_file}")

Moved Customer Contracts.csv to datasets\Customer Contracts.csv
Moved Customer Demo.csv to datasets\Customer Demo.csv
Moved Customer Engagements.csv to datasets\Customer Engagements.csv


In [25]:
data_path = h+'/'+dataset_dir+'/'

df = {}  # Use curly braces to initialize an empty dictionary instead of a list
for file in csv_files:
    try:
        df[file] = pd.read_csv(data_path+file)
    except UnicodeDecodeError:
        df[file] = pd.read_csv(data_path+file, encoding="ISO-8859-1")
    print(file)

Customer Contracts.csv
Customer Demo.csv
Customer Engagements.csv


In [36]:
for k in csv_files:
    dataframe = df[k]
    clean_tbl_name =  k.lower().replace(" ","_").replace("?","") \
                        .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","") \
                        .replace(")","").replace(r"(","").replace("$","")

    # remove.csv extension from clean_tbl_name
    tbl_name = '{0}'.format(clean_tbl_name.split('.')[0])
    print(tbl_name)

    #clean table columns
    dataframe.columns = [x.lower().replace(" ","_").replace("?","") \
                        .replace("-","_").replace(r"/","_").replace("\\","_").replace("%","") \
                        .replace(")","").replace(r"(","").replace("$","") for  x in dataframe.columns]
    

    #replacement dictionary that maps pandas dtypes to sql dtypes
    replacements = {
        'object': 'varchar', 'float64': 'float',
        'int64': 'int',
        'datetime64': 'timestamp',
        'timedelta64 [ns]': 'varchar'
    }
    #table schema
    col_str = ", ".join("{} {}".format (n, d) for (n, d) in zip(dataframe.columns, dataframe.dtypes.replace(replacements))) 
    print(col_str)

    #open a database connection

    host = 'localhost'
    dbname = 'importcsv'
    username = 'postgres'
    password = '1111'
    
    conn_string = "host=%s dbname=%s user=%s password=%s" % (host, dbname, username, password)
    conn = psycopg2.connect (conn_string)
    cursor = conn.cursor()
    print('opened database successfully')

    #drop table with same name 
    cursor.execute("drop table if exists %s;" % (tbl_name))
    
    #create table
    cursor.execute("create table %s (%s);" % (tbl_name, col_str))
    print('{0} was created successfully'.format(tbl_name))
    
    #insert values to table
    
    #save df to csv
    dataframe.to_csv (k, header=dataframe.columns, index=False, encoding='utf-8')
    
    #open the csv file, save it as an object
    my_file = open(k)
    print('file opened in memory')

    #upload to db
    
    SQL_STATEMENT = """
    COPY %s FROM STDIN WITH
        CSV
        HEADER
        DELIMITER AS ','
    """
    
    cursor.copy_expert (sql=SQL_STATEMENT % tbl_name, file=my_file)
    print('file copied to db')

    cursor.execute("grant select on table %s to public" % tbl_name)
    conn.commit()
    
    cursor.close()
    print('table {0} imported to db completed'.format(tbl_name))

#for loop end message
print('all tables have been successfully imported into the db')

customer_contracts
customer_name varchar, start_date varchar, end_date varchar, contract_amount_m float, invoice_sent varchar, paid varchar
opened database successfully
customer_contracts was created successfully
file opened in memory
file copied to db
table customer_contracts imported to db completed
customer_demo
customer_id int, customer_name varchar, employee_count int, office_location varchar
opened database successfully
customer_demo was created successfully
file opened in memory
file copied to db
table customer_demo imported to db completed
customer_engagements
customer_id int, num_of_users int, _of_all_employees varchar, sso varchar, launched varchar
opened database successfully
customer_engagements was created successfully
file opened in memory
file copied to db
table customer_engagements imported to db completed
all tables have been successfully imported into the db
